In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
sid4sal_alpaca_dataset_small_path = kagglehub.dataset_download('sid4sal/alpaca-dataset-small')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader, random_split
import torchvision
from torchvision import datasets, transforms, models
import time
import copy

# -------------------------------
# 1. DATA LOADING (Already Done)
# -------------------------------
data_dir = '/kaggle/input/alpaca-dataset-small/dataset'  # Your path

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Load full dataset
full_dataset = datasets.ImageFolder(root=data_dir, transform=None)

# Split
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_ds, val_ds = random_split(full_dataset, [train_size, val_size])

train_ds.dataset.transform = data_transforms['train']
val_ds.dataset.transform = data_transforms['val']

dataloaders = {
    'train': DataLoader(train_ds, batch_size=16, shuffle=True, num_workers=2, pin_memory=True),
    'val': DataLoader(val_ds, batch_size=16, shuffle=False, num_workers=2, pin_memory=True)
}

dataset_sizes = {'train': len(train_ds), 'val': len(val_ds)}
class_names = full_dataset.classes
print(f"Classes: {class_names}  |  Train: {dataset_sizes['train']}  |  Val: {dataset_sizes['val']}")

# -------------------------------
# 2. DEVICE & MODEL SETUP
# -------------------------------
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load pre-trained ResNet-18
model = models.resnet18(weights='IMAGENET1K_V1')
num_features = model.fc.in_features

# Freeze all layers except the final classifier
for param in model.parameters():
    param.requires_grad = False

# Replace final layer for 2 classes
model.fc = nn.Linear(num_features, 2)
model = model.to(device)

# Loss and optimizer (only train the new fc layer)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.fc.parameters(), lr=0.001, momentum=0.9)
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

# -------------------------------
# 3. TRAINING FUNCTION
# -------------------------------
def train_model(model, criterion, optimizer, scheduler, num_epochs=10):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                # Forward
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # Backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # Deep copy the model if it's the best so far
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:.4f}')

    # Load best model weights
    model.load_state_dict(best_model_wts)
    return model

# -------------------------------
# 4. TRAIN THE MODEL
# -------------------------------
model = train_model(model, criterion, optimizer, scheduler, num_epochs=10)

# -------------------------------
# 5. SAVE THE FINE-TUNED MODEL
# -------------------------------
output_path = '/kaggle/working/fine_tuned_alpaca_model.pth'
torch.save(model.state_dict(), output_path)
print(f"Model saved to: {output_path}")

In [ ]:
import torch
from torchvision import models, transforms
from PIL import Image
import torch.nn.functional as F

# --- CONFIG ---
img_path = "/kaggle/input/alpaca-dataset-small/dataset/not alpaca/028955b87770e194.jpg"
model_path = "/kaggle/working/fine_tuned_alpaca_model.pth"
class_names = ['alpaca', 'not_alpaca']
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- LOAD MODEL ---
model = models.resnet18(weights=None)
model.fc = torch.nn.Linear(model.fc.in_features, 2)
model.load_state_dict(torch.load(model_path, map_location=device))
model.eval().to(device)

# --- TRANSFORM ---
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# --- PREDICT ---
img = Image.open(img_path).convert("RGB")
input_tensor = transform(img).unsqueeze(0).to(device)

with torch.no_grad():
    output = model(input_tensor)
    probs = F.softmax(output, dim=1)[0]
    conf, pred = torch.max(probs, 0)

print(f"Prediction: {class_names[pred]}")
print(f"Confidence: {conf.item():.2f}")